In [ ]:
# from ipywidgets import Dropdown, VBox
from collections import OrderedDict
import os
import pandas as pd

os.chdir("C:/Users/zuk-8/Documents_AZ/Projects/Fanta")

## Global variables

In [ ]:
VOTO_TYPE = 'Voto_Italia'

In [ ]:
df = pd.read_csv('Output_FC/output_fc.csv')

In [ ]:
GAMES_PS = df[['Week', 'Season']].drop_duplicates().groupby('Season').count()
GAMES_PS

## Penalties

In [ ]:
penalties = pd.concat([df.loc[df['Rf']>0], df.loc[df['Rs']>0]])[['Season', 'Rf', 'Rs']].groupby('Season').sum()
penalties['Penalties'] = penalties['Rf'] + penalties['Rs']
penalties['Score_Ratio'] = (penalties['Rf'] / penalties['Penalties']) * 100
penalties['Points'] = 3*penalties['Rf'] -3*penalties['Rs']
penalties.astype(int)

## Grades

In [ ]:
def get_bonus(row, penalty=True):
    goals = 3*row['Gf'] - row['Gs'] - 2*row['Au']
    penalties = 3*row['Rp'] - 3*row['Rs'] + 3*row['Rf']
    assists = row['Ass'] + row['Asf']
    malus = - 0.5*row['Amm'] - row['Esp']
    if penalty is True:
        return goals + penalties + assists + malus
    elif penalty is False:
        return goals + assists + malus
    else:
        raise ValueError("penalty must be either True or False")
        return None


def get_bonus_nop(row):
    return get_bonus(row, penalty=False)


In [ ]:
# Add bonuses
df['Bonus'] = df.apply(get_bonus, axis=1)
df['Bonus_nop'] = df.apply(get_bonus_nop, axis=1)

In [ ]:
# Stats per role
avg_grade_role = df.set_index('Ruolo')[[VOTO_TYPE,'Bonus_nop']].sum(1).reset_index().groupby('Ruolo').mean()[0].round(2)
std_grade_role = df.set_index('Ruolo')[[VOTO_TYPE,'Bonus_nop']].sum(1).reset_index().groupby('Ruolo').std()[0].round(2)
# avg_grade_role = df.groupby(['Ruolo'])[['Voto_Italia','Bonus_nop']].mean().sum(1).round(2)
avg_grade_role_season = df.groupby(['Ruolo', 'Season'])[['Voto_Italia','Bonus_nop']].mean().sum(1).unstack().round(2)
REPLACEMENT_PER_ROLE = (avg_grade_role - 0.5 * std_grade_role).round(2)
print('Average grades per role')
print(avg_grade_role)
print('\nAverage grades per role for replacements')
print(REPLACEMENT_PER_ROLE)
print('Average grades per role by season')
print(avg_grade_role_season)

In [ ]:
def compute_score(raw_stats, expected_games=None, replacement_score=None):
    if expected_games is None:
        expected_games = sum([GAMES_PS.loc[season, 'Week'] for season in raw_stats['Season'].unique()])
    if replacement_score is None:
        role = raw_stats.groupby('Ruolo')['Week'].count().sort_values(ascending=False).index[0]
        replacement_score = REPLACEMENT_PER_ROLE[role]
    games_played = len(raw_stats.index)
    if games_played > expected_games:
        raise ValueError(f"games_played ({games_played}) > expected_games ({expected_games})")
    sum_grades = raw_stats[[VOTO_TYPE,'Bonus_nop']].sum(1)
    return ((sum_grades.sum() + (expected_games-games_played) * replacement_score) / expected_games).round(2)

## Player class

In [ ]:
def search_id_from_name(name):
    pass


def get_player_id_from_name(name):
    ids = [int(x) for x in df[df['Nome']==name.upper()]['Fantacalcio_id'].unique()]
    if len(ids) == 1:
        return ids[0]
    elif len(ids) == 0:
        raise ValueError(f"Couldn't find any grades for player {name}")
        # search_id_from_name(name)
    elif len(ids) > 1:
        raise ValueError(f"Found more than 1 ID for player {name}: {ids}")
    else:
        raise ValueError("Unclear error")


def get_season_week_str(row):
    season = row['Season']
    week = row['Week']
    return f"{str(int(season))}-{str(int(season)-1999)} week {week:02d}"


def get_first_last_team(team):
    team = team.sort_values(by=['Season', 'Week'])
    team_name = team.iloc[0]['Team']
    first = get_season_week_str(team.iloc[0])
    for row in team.index:
        if team.loc[row]['Team'] == team_name:
            last = get_season_week_str(team.iloc[0])
            team = team.drop(index=row, axis=1)
        else:
            return f"{team_name} from {first} to {last}", team
    return f"{team_name} from {first} to {last}", None


def get_teams_from_fcid(fc_id):
    team = df.loc[df['Fantacalcio_id']==fc_id, ['Team', 'Week', 'Season']]
#     team.groupby(['Season', 'Team'])['Week'].agg(['min', 'max'])
    teams = list()
    while team is not None:
        first_last, team = get_first_last_team(team)
        teams.append(first_last)
    return teams


def get_roles_from_fcid(fc_id):
    role = df.loc[df['Fantacalcio_id']==fc_id, ['Ruolo', 'Week', 'Season']]
    return role.groupby('Season').first()['Ruolo']


def get_raw_stats_from_fcid(fc_id):
    stats = df.loc[df['Fantacalcio_id']==fc_id].copy()
    return stats


def player_stats(raw_stats):
    stats = OrderedDict([])
    stats['Game'] = len(raw_stats.index)
    stats['Score'] = compute_score(raw_stats)
    for col in raw_stats.columns:
        if any([col.startswith('Voto_'), col.startswith('Bonus')]):
            stats[col] = raw_stats[col].mean()
    stats['Bonus'] = raw_stats['Bonus'].mean()
    stats['Bonus_nop'] = raw_stats['Bonus_nop'].mean()
    if any(raw_stats['Ruolo']=='P'):
        stats['Goal_Conceded'] = raw_stats['Gs'].sum()
        stats['Penalty_Saved'] = raw_stats['Rp'].sum()
    elif all(raw_stats['Ruolo']!='P'):
        stats['Goal_Scored'] = raw_stats['Gf'].sum()
        stats['Penalty_Scored'] = raw_stats['Rf'].sum()
        stats['Penalty_Missed'] = raw_stats['Rs'].sum()
        stats['Assist'] = raw_stats['Ass'].sum() + raw_stats['Asf'].sum()
    else:
        raise ValueError("Role is unclear: 'P' for some but not all games")
    stats['Own_Goal'] = raw_stats['Au'].sum()
    stats['Red_Card'] = raw_stats['Esp'].sum()
    stats['Yellow_Card'] = raw_stats['Amm'].sum()
    return stats


def analyse_stats(raw_stats):
    # Total
    stats = player_stats(raw_stats)
    all_stats = pd.Series(stats).to_frame('Total')
    all_stats['Avg'] = all_stats['Total'].iloc[7:] / stats['Game']
    # Season
    seasons = raw_stats['Season'].unique()[::-1]
    stats_py = OrderedDict([(season, player_stats(raw_stats.loc[df['Season']==season])) for season in seasons])
    all_stats = all_stats.join(pd.DataFrame(stats_py), how='right')
    # Team
    teams = raw_stats['Team'].iloc[::-1].unique()
    stats_team = OrderedDict([(team, player_stats(raw_stats.loc[df['Team']==team])) for team in teams])
    all_stats = all_stats.join(pd.DataFrame(stats_team))
    # Role
    ruolo = raw_stats.groupby('Season').first()[['Ruolo']].T
    # Concat
    all_stats = pd.concat([all_stats.round(2), ruolo])
    all_stats = all_stats[['Total'] + [t for t in teams] + [s for s in seasons] + ['Avg']]
    return all_stats


name = 'Gomez'

fc_id = get_player_id_from_name(name)
teams = get_teams_from_fcid(fc_id)
roles = get_roles_from_fcid(fc_id)
raw_stats = get_raw_stats_from_fcid(fc_id)
stats = analyse_stats(raw_stats)
print("Teams:\n"+"/n".join(teams))
print("\nStats")
stats.fillna('-')